## Load Base Stable Diffusion Model

In [ ]:
from diffusers import StableDiffusionImg2ImgPipeline
from pruna.algorithms.smasher_config import AutoSmasherConfig
from pruna.smash import smash
import torch
import requests
from PIL import Image
from io import BytesIO

from diffusers import DiffusionPipeline
import torch

pipe = DiffusionPipeline.from_pretrained("stabilityai/stable-diffusion-xl-base-1.0", torch_dtype=torch.float16, use_safetensors=True, variant="fp16")
pipe.unet.load_attn_procs('hunz/stable-diffusion-xl-lora')

pipe.to("cuda")

prompt = "Iron Man flying full speed with blue flames"

## Smash it!

In [ ]:
from pruna.algorithms.smasher_config import SmasherConfig

smasher_config = SmasherConfig()
smasher_config['compiler'] = 'diffusers'
smasher_config['max_batch_size'] = 1
smasher_config['image_height'] = 1024
smasher_config['image_width'] = 1024
smasher_config['version'] = 'xl-1.0'
smasher_config['scheduler'] = "DDIM"

In [ ]:
smashed_model = smash(
        model=pipe,
        data_module="LAION256",
        api_key='your-api-key',
        model_config=None,
        smasher_config=smasher_config,
        device='cuda',
    )

## Base Model Generation

In [ ]:
%%time
pipe(prompt=prompt).images[0].show()

## Smashed Model Generation

In [ ]:
%%time
smashed_model(prompt, image_height=1024, image_width=1024)[0]